In [1]:
import pandas as pd
import numpy as np

In [2]:
deldata=pd.read_csv("deliveries.csv")

In [3]:
deldata.shape

(179078, 21)

In [4]:
print("No of Matches:",len(deldata["match_id"].unique()))

No of Matches: 756


In [5]:
deldata.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [6]:
match=pd.read_csv("matches.csv")

In [7]:
match.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


# Score Prediction

## Preprocessing

In [8]:
#getting dummy variables for winning teams
match["team1_win"]=0
match.loc[match["team1"]==match["winner"],"team1_win"]=1

match["team2_win"]=0
match.loc[match["team2"]==match["winner"],"team2_win"]=1

In [9]:
matchwin=match[["id","team1","team1_win"]]

In [10]:
matchwin=matchwin.rename(columns={"team1":"team","team1_win":"win"})

In [11]:
matchwin

,id,team,win
0,1,Sunrisers Hyderabad,1
1,2,Mumbai Indians,0
2,3,Gujarat Lions,0
3,4,Rising Pune Supergiant,0
4,5,Royal Challengers Bangalore,1
...,...,...,...
751,11347,Kolkata Knight Riders,0
752,11412,Chennai Super Kings,0
753,11413,Sunrisers Hyderabad,0
754,11414,Delhi Capitals,0


In [12]:
matchwin2=match[["id","team2","team2_win"]].rename(columns={"team2":"team","team2_win":"win"})

In [13]:
matchwin2

,id,team,win
0,1,Royal Challengers Bangalore,0
1,2,Rising Pune Supergiant,1
2,3,Kolkata Knight Riders,1
3,4,Kings XI Punjab,1
4,5,Delhi Daredevils,0
...,...,...,...
751,11347,Mumbai Indians,1
752,11412,Mumbai Indians,1
753,11413,Delhi Capitals,1
754,11414,Chennai Super Kings,1


In [14]:
matchwin=matchwin.append(matchwin2)

In [15]:
matchwin

,id,team,win
0,1,Sunrisers Hyderabad,1
1,2,Mumbai Indians,0
2,3,Gujarat Lions,0
3,4,Rising Pune Supergiant,0
4,5,Royal Challengers Bangalore,1
...,...,...,...
751,11347,Mumbai Indians,1
752,11412,Mumbai Indians,1
753,11413,Delhi Capitals,1
754,11414,Chennai Super Kings,1


In [16]:
grps=deldata.groupby(["match_id","inning"])

In [17]:
def getscores(df,sums,counts):
    gdf=df.groupby("over")
    dat={}
    var={}
    for x in sums+counts:
        var[x]=0    
    for x,y in gdf:
        for z in sums:
            l=str(str(x)+"_"+str(z))
            var[z]=var[z]+int(y[z].sum())
            dat[l]=var[z]
        for z in counts:
            l=str(str(x)+"_"+str(z))
            var[z]=var[z]+int(y[z].count())
            dat[l]=var[z]          
    return dat
        

In [18]:
data=[]
for x,y in grps:
    dat=getscores(y,["total_runs"],["player_dismissed"])
    dat["match_id"]=x[0]
    dat["inning"]=x[1]
    data.append(dat)    

In [19]:
df=pd.DataFrame(data)

In [20]:
df.head()

,1_total_runs,1_player_dismissed,2_total_runs,2_player_dismissed,3_total_runs,3_player_dismissed,4_total_runs,4_player_dismissed,5_total_runs,5_player_dismissed,...,17_total_runs,17_player_dismissed,18_total_runs,18_player_dismissed,19_total_runs,19_player_dismissed,20_total_runs,20_player_dismissed,match_id,inning
0,7,0,23.0,1.0,29.0,1.0,33.0,1.0,42.0,1.0,...,162.0,3.0,179.0,3.0,191.0,4.0,207.0,4.0,1,1
1,11,0,12.0,0.0,28.0,0.0,43.0,0.0,48.0,0.0,...,154.0,6.0,156.0,8.0,165.0,9.0,172.0,10.0,1,2
2,11,0,19.0,0.0,28.0,0.0,41.0,0.0,48.0,1.0,...,137.0,6.0,145.0,6.0,154.0,7.0,184.0,8.0,2,1
3,8,0,23.0,0.0,35.0,0.0,39.0,1.0,43.0,1.0,...,158.0,3.0,165.0,3.0,172.0,3.0,187.0,3.0,2,2
4,8,0,16.0,0.0,22.0,0.0,29.0,1.0,40.0,1.0,...,141.0,3.0,153.0,3.0,169.0,3.0,183.0,4.0,3,1


In [21]:
newdf=df[df.isnull().any(axis=1)==False]

In [22]:
x=newdf[["5_total_runs","5_player_dismissed","10_total_runs","10_player_dismissed","15_total_runs","15_player_dismissed",'inning']]

In [23]:
y=newdf["20_total_runs"]

In [24]:
x.to_csv("xx.csv",index=False)

In [25]:
y.to_csv("yy.csv",index=False,header=["y"])

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=42)

### Linear Regression

In [28]:
from sklearn.linear_model import LinearRegression

In [29]:
model=LinearRegression()

In [30]:
model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [31]:
predictions=model.predict(X_test)

In [32]:
total_error=0
total_inns=0
for i,j in zip(predictions,y_test):
    total_error=total_error+abs(i-j)
    total_inns=total_inns+1
    
    

In [33]:
total_error/total_inns

10.264784527445885

In [34]:
print(model.intercept_, model.coef_)

49.5461504063321 [-0.09291575  0.05520597 -0.14721753  0.1296244   1.21769322 -1.74231469
 -3.73625344]


### Random Forest Regression

In [35]:
from sklearn.ensemble import RandomForestRegressor

In [36]:
regr = RandomForestRegressor(max_depth=10, random_state=0,
                             n_estimators=100)
regr.fit(X_train, y_train)  

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [37]:
predictions=regr.predict(X_test)

In [38]:
total_error=0
total_inns=0
for i,j in zip(predictions,y_test):
    total_error=total_error+abs(i-j)
    total_inns=total_inns+1
print("Mean Absolute Error:",total_error/total_inns)

Mean Absolute Error: 10.945068739988615


### Support Vector Regression

In [39]:
from sklearn.svm import SVR

In [40]:
clf = SVR(gamma='scale', C=1, epsilon=0.2)
clf.fit(X_train, y_train) 

SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [41]:
predictions=clf.predict(X_test)

In [42]:
total_error=0
total_inns=0
for i,j in zip(predictions,y_test):
    total_error=total_error+abs(i-j)
    total_inns=total_inns+1
print("Mean Absolute Error:",total_error/total_inns)

Mean Absolute Error: 12.313418711263417


### MLP Regression

In [43]:
from sklearn.neural_network import MLPRegressor

In [44]:
reg=MLPRegressor(max_iter=10000,hidden_layer_sizes=(10,30,10))

In [45]:
reg.fit(X_train,y_train)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(10, 30, 10), learning_rate='constant',
             learning_rate_init=0.001, max_iter=10000, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [46]:
predictions=reg.predict(X_test)

In [47]:
total_error=0
total_inns=0
for i,j in zip(predictions,y_test):
    total_error=total_error+abs(i-j)
    total_inns=total_inns+1
print("Mean Absolute Error:",total_error/total_inns)

Mean Absolute Error: 11.326744821376444


# Win Prediction - Half Way

## Preprocessing

In [48]:
match["team1_win"]=0
match.loc[match["team1"]==match["winner"],"team1_win"]=1

In [49]:
match["team2_win"]=0
match.loc[match["team2"]==match["winner"],"team2_win"]=1


In [50]:
match["team1_toss_win"]=0
match.loc[match["team1"]==match["toss_winner"],"team1_toss_win"]=1

In [51]:
match["team2_toss_win"]=0
match.loc[match["team2"]==match["toss_winner"],"team2_toss_win"]=1

In [52]:
tm1in1=[]
tm2in1=[]
for x,y,z,i in zip(match["team1"].values,match["team2"].values,match["toss_winner"].values,match["toss_decision"].values):
    if x==z:
        tm1in1.append(i)
        if i=="field":
            tm2in1.append("bat")
        else:
            tm2in1.append("field")
    else:
        tm2in1.append(i)
        if i=="field":
            tm1in1.append("bat")
        else:
            tm1in1.append("field")
        
    

In [53]:
match["team1_inn1"]=tm1in1
match["team2_inn1"]=tm2in1

In [54]:
match.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,...,venue,umpire1,umpire2,umpire3,team1_win,team2_win,team1_toss_win,team2_toss_win,team1_inn1,team2_inn1
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,...,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN,1,0,0,1,bat,field
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,...,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN,0,1,0,1,bat,field
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,...,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN,0,1,0,1,bat,field
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,...,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN,0,1,0,1,bat,field
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,...,M Chinnaswamy Stadium,NaN,NaN,NaN,1,0,1,0,bat,field


In [55]:
matchwin=match[["id","team1","team1_win","team1_toss_win","team1_inn1","venue"]]

matchwin=matchwin.rename(columns={"team1":"team","team1_win":"win","team1_toss_win":"toss_win","team1_inn1":"inn1"})

matchwin2=match[["id","team2","team2_win","team2_toss_win","team2_inn1","venue"]].rename(columns={"team2":"team","team2_win":"win","team2_toss_win":"toss_win","team2_inn1":"inn1"})

matchwin=matchwin.append(matchwin2)

In [56]:
matchwin

,id,team,win,toss_win,inn1,venue
0,1,Sunrisers Hyderabad,1,0,bat,"Rajiv Gandhi International Stadium, Uppal"
1,2,Mumbai Indians,0,0,bat,Maharashtra Cricket Association Stadium
2,3,Gujarat Lions,0,0,bat,Saurashtra Cricket Association Stadium
3,4,Rising Pune Supergiant,0,0,bat,Holkar Cricket Stadium
4,5,Royal Challengers Bangalore,1,1,bat,M Chinnaswamy Stadium
...,...,...,...,...,...,...
751,11347,Mumbai Indians,1,1,field,Wankhede Stadium
752,11412,Mumbai Indians,1,0,field,M. A. Chidambaram Stadium
753,11413,Delhi Capitals,1,1,field,ACA-VDCA Stadium
754,11414,Chennai Super Kings,1,1,field,ACA-VDCA Stadium


In [57]:
matchwin=matchwin.sort_values(by=["id"])

In [58]:
matchwin

,id,team,win,toss_win,inn1,venue
0,1,Sunrisers Hyderabad,1,0,bat,"Rajiv Gandhi International Stadium, Uppal"
0,1,Royal Challengers Bangalore,0,1,field,"Rajiv Gandhi International Stadium, Uppal"
1,2,Rising Pune Supergiant,1,1,field,Maharashtra Cricket Association Stadium
1,2,Mumbai Indians,0,0,bat,Maharashtra Cricket Association Stadium
2,3,Gujarat Lions,0,0,bat,Saurashtra Cricket Association Stadium
...,...,...,...,...,...,...
753,11413,Sunrisers Hyderabad,0,0,bat,ACA-VDCA Stadium
754,11414,Chennai Super Kings,1,1,field,ACA-VDCA Stadium
754,11414,Delhi Capitals,0,0,bat,ACA-VDCA Stadium
755,11415,Mumbai Indians,1,1,bat,Rajiv Gandhi Intl. Cricket Stadium


In [59]:
mv=match[["id","venue"]]

md=mv.set_index("id").join(deldata.set_index("match_id"))

md=md[md["inning"].isin([1,2])]

l=md.groupby(["venue",md.index.get_level_values(0),"inning"]).agg("sum")

means=l.groupby(["venue","inning"]).agg(["mean"])["total_runs"]

In [60]:
means=means.reset_index(level=[0,1])

In [61]:
x=means.groupby(["venue","inning"])

In [62]:
dc={}
for x,y,z in zip(means["venue"].values,means["inning"].values,means["mean"].values) :
    if y==1:
        dc[x]={"one":z}
    else:
        dc[x]["two"]=z
        
            
    

In [63]:
dc

{'ACA-VDCA Stadium': {'one': 163.0, 'two': 166.0},
 'Barabati Stadium': {'one': 167.71428571428572, 'two': 157.71428571428572},
 'Brabourne Stadium': {'one': 180.36363636363637, 'two': 168.9090909090909},
 'Buffalo Park': {'one': 147.33333333333334, 'two': 119.0},
 'De Beers Diamond Oval': {'one': 158.0, 'two': 141.0},
 'Dr DY Patil Sports Academy': {'one': 146.64705882352942,
  'two': 136.2941176470588},
 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium': {'one': 158.0,
  'two': 125.72727272727273},
 'Dubai International Cricket Stadium': {'one': 148.85714285714286,
  'two': 146.0},
 'Eden Gardens': {'one': 161.62337662337663, 'two': 148.2077922077922},
 'Feroz Shah Kotla': {'one': 164.4179104477612, 'two': 149.6969696969697},
 'Feroz Shah Kotla Ground': {'one': 162.85714285714286,
  'two': 157.57142857142858},
 'Green Park': {'one': 161.25, 'two': 163.25},
 'Himachal Pradesh Cricket Association Stadium': {'one': 175.66666666666666,
  'two': 146.22222222222223},
 'Holkar Cricket S

In [64]:
inn1=[]
inn2=[]
for x in matchwin["venue"].values:
    inn1.append(dc[x]["one"])
    inn2.append(dc[x]["two"])

In [65]:
matchwin["inn1_avg"]=inn1
matchwin["inn2_avg"]=inn2

In [66]:
matchwin['inning']=2
matchwin.loc[matchwin["inn1"]=="bat","inning"]=1

In [67]:
matchwin.head()

,id,team,win,toss_win,inn1,venue,inn1_avg,inn2_avg,inning
0,1,Sunrisers Hyderabad,1,0,bat,"Rajiv Gandhi International Stadium, Uppal",156.589286,148.017857,1
0,1,Royal Challengers Bangalore,0,1,field,"Rajiv Gandhi International Stadium, Uppal",156.589286,148.017857,2
1,2,Rising Pune Supergiant,1,1,field,Maharashtra Cricket Association Stadium,170.952381,156.238095,2
1,2,Mumbai Indians,0,0,bat,Maharashtra Cricket Association Stadium,170.952381,156.238095,1
2,3,Gujarat Lions,0,0,bat,Saurashtra Cricket Association Stadium,168.900000,162.700000,1


In [68]:
final = pd.merge(matchwin, df,  how='left', left_on=['id','inning'], right_on = ['match_id','inning'])

In [69]:
v1_df=final[final["inning"]==1]

In [70]:
v2_df=v1_df.dropna()

In [71]:
v2_df=v2_df.reset_index()

In [72]:
v2_df.head()

,index,id,team,win,toss_win,inn1,venue,inn1_avg,inn2_avg,inning,...,16_player_dismissed,17_total_runs,17_player_dismissed,18_total_runs,18_player_dismissed,19_total_runs,19_player_dismissed,20_total_runs,20_player_dismissed,match_id
0,0,1,Sunrisers Hyderabad,1,0,bat,"Rajiv Gandhi International Stadium, Uppal",156.589286,148.017857,1,...,3.0,162.0,3.0,179.0,3.0,191.0,4.0,207.0,4.0,1.0
1,3,2,Mumbai Indians,0,0,bat,Maharashtra Cricket Association Stadium,170.952381,156.238095,1,...,6.0,137.0,6.0,145.0,6.0,154.0,7.0,184.0,8.0,2.0
2,4,3,Gujarat Lions,0,0,bat,Saurashtra Cricket Association Stadium,168.900000,162.700000,1,...,3.0,141.0,3.0,153.0,3.0,169.0,3.0,183.0,4.0,3.0
3,6,4,Rising Pune Supergiant,0,0,bat,Holkar Cricket Stadium,163.444444,161.000000,1,...,4.0,128.0,4.0,133.0,5.0,147.0,5.0,163.0,6.0,4.0
4,8,5,Royal Challengers Bangalore,1,1,bat,M Chinnaswamy Stadium,170.575342,149.847222,1,...,4.0,142.0,6.0,144.0,6.0,150.0,6.0,157.0,8.0,5.0


In [73]:
v3=v2_df.groupby("team")["win"].rolling(3).sum()

In [74]:
v3=v3.reset_index(level=[0,1])

In [75]:
v2_df["roll_win"]=0

In [76]:
v2_df.head()

,index,id,team,win,toss_win,inn1,venue,inn1_avg,inn2_avg,inning,...,17_total_runs,17_player_dismissed,18_total_runs,18_player_dismissed,19_total_runs,19_player_dismissed,20_total_runs,20_player_dismissed,match_id,roll_win
0,0,1,Sunrisers Hyderabad,1,0,bat,"Rajiv Gandhi International Stadium, Uppal",156.589286,148.017857,1,...,162.0,3.0,179.0,3.0,191.0,4.0,207.0,4.0,1.0,0
1,3,2,Mumbai Indians,0,0,bat,Maharashtra Cricket Association Stadium,170.952381,156.238095,1,...,137.0,6.0,145.0,6.0,154.0,7.0,184.0,8.0,2.0,0
2,4,3,Gujarat Lions,0,0,bat,Saurashtra Cricket Association Stadium,168.900000,162.700000,1,...,141.0,3.0,153.0,3.0,169.0,3.0,183.0,4.0,3.0,0
3,6,4,Rising Pune Supergiant,0,0,bat,Holkar Cricket Stadium,163.444444,161.000000,1,...,128.0,4.0,133.0,5.0,147.0,5.0,163.0,6.0,4.0,0
4,8,5,Royal Challengers Bangalore,1,1,bat,M Chinnaswamy Stadium,170.575342,149.847222,1,...,142.0,6.0,144.0,6.0,150.0,6.0,157.0,8.0,5.0,0


In [77]:
for x,y in zip(v3["level_1"].values,v3["win"].values):
    v2_df.loc[x,"roll_win"]=y
   

In [78]:
v2_df["roll_win"]=v2_df["roll_win"]-v2_df["win"]

In [79]:
v3_df=v2_df.dropna()

In [80]:
v3_df["roll_win"].max()

2.0

In [81]:
y=v3_df["win"]
x=v3_df[["toss_win","5_total_runs","10_total_runs","15_total_runs","20_total_runs","20_player_dismissed","inn1_avg","roll_win"]]

In [82]:
x.to_csv("xx2.csv",index=False)
y.to_csv("yy2.csv",index=False,header=["y"])

### Logistic Regression

In [83]:
from sklearn.linear_model import LogisticRegression

In [84]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=43)

In [85]:
clf = LogisticRegression(random_state=0, solver='lbfgs',max_iter=10000).fit(X_train, y_train)

In [86]:
from sklearn.model_selection import cross_val_score
clf = LogisticRegression(random_state=50, solver='lbfgs')
print(cross_val_score(clf,x,y,cv=5))

[0.65942029 0.7080292  0.71532847 0.66176471 0.60294118]


/Users/jkachhadia/Documents/da/env/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [87]:
clf.fit(x,y)
preds=clf.predict(X_test)

In [88]:
from sklearn.metrics import confusion_matrix

In [89]:
confusion_matrix(y_test, preds)

array([[87, 29],
       [27, 63]])

In [90]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,preds)

0.7281553398058253

### Random Forest Classifier

In [91]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=10,
                             random_state=0)
clf.fit(X_train, y_train)
preds=clf.predict(X_test)

In [92]:
print("Confusion Matrix")
print(confusion_matrix(y_test, preds))
print("Accuracy")

print(accuracy_score(y_test,preds))

Confusion Matrix
[[85 31]
 [31 59]]
Accuracy
0.6990291262135923


### Neural Networks

In [93]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(50,55,50),
                             random_state=0,max_iter=2000)
clf.fit(X_train, y_train)
preds=clf.predict(X_test)

In [94]:
print("Confusion Matrix")
print(confusion_matrix(y_test, preds))
print("Accuracy")

print(accuracy_score(y_test,preds))

Confusion Matrix
[[84 32]
 [25 65]]
Accuracy
0.7233009708737864


# Win Prediction - 10 Overs Left

## Preprocessing

In [95]:
v1=final[final["inning"]==1]
v2=final[final["inning"]==2]

In [96]:
new_f=pd.merge(v1, v2,  how='left', left_on=['id'], right_on = ['id'])

In [97]:
newdata=new_f[["id","team_x","win_x","toss_win_x","inn1_avg_x","5_total_runs_x","5_total_runs_y","10_total_runs_x","10_total_runs_y","10_player_dismissed_y","15_total_runs_y","15_total_runs_x","20_total_runs_x","20_player_dismissed_x"]]

In [98]:
newdata10=newdata.dropna()

In [99]:
newdata10=newdata10.reset_index()
v3=newdata10.groupby("team_x")["win_x"].rolling(3).sum()

v3=v3.reset_index(level=[0,1])

newdata10["roll_win"]=0

for x,y in zip(v3["level_1"].values,v3["win_x"].values):
    newdata10.loc[x,"roll_win"]=y
    

In [100]:
nd=newdata10.dropna()

In [101]:
nd["roll_win"]=nd["roll_win"]-nd["win_x"]

/Users/jkachhadia/Documents/da/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [102]:
nd.head()

,index,id,team_x,win_x,toss_win_x,inn1_avg_x,5_total_runs_x,5_total_runs_y,10_total_runs_x,10_total_runs_y,10_player_dismissed_y,15_total_runs_y,15_total_runs_x,20_total_runs_x,20_player_dismissed_x,roll_win
11,11,12,Royal Challengers Bangalore,0,0,170.575342,34.0,18.0,66.0,48.0,5.0,91.0,110.0,142.0,5.0,1.0
15,15,16,Gujarat Lions,0,0,167.876712,31.0,47.0,75.0,89.0,2.0,132.0,120.0,176.0,4.0,0.0
16,16,17,Rising Pune Supergiant,1,0,170.575342,50.0,39.0,79.0,67.0,2.0,92.0,117.0,161.0,8.0,0.0
17,17,18,Delhi Daredevils,0,1,164.417910,50.0,34.0,83.0,88.0,3.0,135.0,110.0,168.0,7.0,2.0
18,18,19,Sunrisers Hyderabad,1,0,156.589286,25.0,32.0,54.0,62.0,5.0,104.0,108.0,159.0,6.0,1.0


In [103]:
y=nd["win_x"]
x=nd[["toss_win_x","5_total_runs_x","10_total_runs_x","15_total_runs_x","20_total_runs_x","5_total_runs_y","10_total_runs_y","10_player_dismissed_y","inn1_avg_x","roll_win"]]

In [104]:
x.to_csv("x3.csv",index=False)
y.to_csv("y3.csv",index=False,header=["y"])

### Logistic Regression

In [105]:
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=42)

clf = LogisticRegression(random_state=0, solver='lbfgs',max_iter=10000).fit(X_train, y_train)

preds=clf.predict(X_test)

print("Confusion Matrix")
print(confusion_matrix(y_test, preds))
print("Accuracy")
print(accuracy_score(y_test,preds))

Confusion Matrix
[[73 17]
 [28 78]]
Accuracy
0.7704081632653061


### Random Forest Classifier

In [106]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=10,
                             random_state=0)
clf.fit(X_train, y_train)
preds=clf.predict(X_test)

print("Confusion Matrix")
print(confusion_matrix(y_test, preds))
print("Accuracy")
print(accuracy_score(y_test,preds))

Confusion Matrix
[[70 20]
 [30 76]]
Accuracy
0.7448979591836735


### Neural Networks

In [107]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(75,75,75),
                             random_state=0,max_iter=2000)
clf.fit(X_train, y_train)
preds=clf.predict(X_test)

print("Confusion Matrix")
print(confusion_matrix(y_test, preds))
print("Accuracy")
print(accuracy_score(y_test,preds))

Confusion Matrix
[[73 17]
 [25 81]]
Accuracy
0.7857142857142857
